In [54]:
import inspect
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import benz_Symm,alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis='sto-3g',symmetry = 'D6v') #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})

In [22]:
BenzHF=scf.RHF(Benz)

In [23]:
%time BenzHF.scf()#dm0=ldm)

converged SCF energy = -227.890711338274
CPU times: user 1.44 s, sys: 39.3 ms, total: 1.48 s
Wall time: 501 ms


-227.8907113382739

In [52]:
%time d1,d2,d3= alch_deriv(BenzHF,[[0],[1]])
d1,d2,d3

[[0], [1]]
CPU times: user 383 ms, sys: 28.3 ms, total: 412 ms
Wall time: 139 ms


(-14.554808443274148, 1.079627074618514, 0.26021645259667053)

In [105]:
dl=1e-3
fdhf1=scf.RHF(FcM(fcs=[[0,3],[dl,dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e1=fdhf1.scf()
fdhf2=scf.RHF(FcM(fcs=[[0,3],[-dl,-dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e2=fdhf2.scf()
e0=BenzHF.scf()

converged SCF energy = -227.890711338274


In [106]:
fdhf3=scf.RHF(FcM(fcs=[[0,3],[2*dl,2*dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e3=fdhf3.scf()
fdhf4=scf.RHF(FcM(fcs=[[0,3],[-2*dl,-2*dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
e4=fdhf4.scf()

In [107]:
(e1-e2)/2/dl,(e1+e2-2*e0)/dl**2,(e3/2-e1+e2-e4/2)/dl**3

(-15.697750917468056, -1.208235630656418, 0.1807762828320847)

In [111]:
%time d1,d2,d3= alch_deriv(BenzHF,[[0,3],[1,1]])
d1,d2,d3

[[0, 3], [1, 1]]
CPU times: user 372 ms, sys: 28 ms, total: 400 ms
Wall time: 129 ms


(-15.697750941059958, -1.2082383098489926, 0.1807643237673)

In [8]:
%time af1= aaff(BenzHF,[1,0,0,0,0,0,0,0,0,0,0,0,0])

CPU times: user 11.2 s, sys: 247 ms, total: 11.4 s
Wall time: 8.53 s


In [9]:
Benz.symmetry

'D6v'

In [10]:
from AP_class import DeltaV,rotate_matrix,parse_charge,alchemy_cphf_deriv,make_dP

In [23]:
from AP_class import rotate_grad,rotate_matrix

In [24]:
g=BenzHF.Gradients().grad_nuc()
g

array([[ 2.96057623e-12, -8.58254608e+00,  7.79752832e-08],
       [ 2.76043077e-13, -2.54741647e+00, -1.14081458e-09],
       [-7.43270294e+00, -4.29127305e+00,  1.21872018e-07],
       [-2.20612738e+00, -1.27370824e+00, -7.74666156e-09],
       [-7.43270294e+00,  4.29127305e+00, -9.64438761e-08],
       [-2.20612738e+00,  1.27370824e+00, -5.62362280e-08],
       [-2.33539577e-12,  8.58254608e+00,  6.95524577e-08],
       [-1.72069303e-12,  2.54741647e+00,  2.13607535e-08],
       [ 7.43270294e+00,  4.29127305e+00,  4.55789859e-08],
       [ 2.20612738e+00,  1.27370824e+00, -8.18970519e-09],
       [ 7.43270294e+00, -4.29127305e+00, -1.42698422e-07],
       [ 2.20612738e+00, -1.27370824e+00, -2.38837909e-08]])

In [25]:
rotate_grad(g,2)

array([[ 7.43270294e+00, -4.29127305e+00, -1.42698422e-07],
       [ 2.20612738e+00, -1.27370824e+00, -2.38837909e-08],
       [ 2.96057623e-12, -8.58254608e+00,  7.79752832e-08],
       [ 2.76043077e-13, -2.54741647e+00, -1.14081458e-09],
       [-7.43270294e+00, -4.29127305e+00,  1.21872018e-07],
       [-2.20612738e+00, -1.27370824e+00, -7.74666156e-09],
       [-7.43270294e+00,  4.29127305e+00, -9.64438761e-08],
       [-2.20612738e+00,  1.27370824e+00, -5.62362280e-08],
       [-2.33539577e-12,  8.58254608e+00,  6.95524577e-08],
       [-1.72069303e-12,  2.54741647e+00,  2.13607535e-08],
       [ 7.43270294e+00,  4.29127305e+00,  4.55789859e-08],
       [ 2.20612738e+00,  1.27370824e+00, -8.18970519e-09]])

In [14]:
from scipy.spatial.transform import Rotation as R

In [15]:
rv=-np.asarray([0,0,1])*np.pi*2/3
r = R.from_rotvec(rv)

In [16]:
np.allclose(rotate_grad(r.apply(g),4),g,atol=1e-6)

True

In [17]:
np.set_printoptions(precision=3,suppress=True)

## Symmetry operations for Benzene $D_{6h}$ axis 
I need to <br>
1) dP (i) -> rotate dP (0/1) <br>
2) af(i) = rotate af (0/1)<br>
3) dV(i) =rotate dV(0/1)<br>

In [18]:
import itertools

In [19]:
from AP_class import APDFT_perturbator

In [20]:
AP=APDFT_perturbator(BenzHF,sites=[1,2,0])

In [21]:
AP.af(3)

No AF found for atom 1. Calculating it from code


array([[-0.02 , -0.035, -0.   ],
       [-0.003, -0.003, -0.   ],
       [ 0.179,  0.103,  0.   ],
       [-0.1  , -0.058, -0.   ],
       [-0.04 ,  0.   , -0.   ],
       [-0.004, -0.001, -0.   ],
       [ 0.003, -0.005, -0.   ],
       [-0.005, -0.001, -0.   ],
       [-0.005, -0.003,  0.   ],
       [ 0.003,  0.002, -0.   ],
       [-0.003,  0.005, -0.   ],
       [-0.004, -0.003, -0.   ]])

In [22]:
AP.af(1),AP.af(7)

(array([[-0.   ,  0.206,  0.   ],
        [ 0.   , -0.115, -0.   ],
        [-0.021, -0.035, -0.   ],
        [-0.001, -0.004, -0.   ],
        [ 0.005, -0.   , -0.   ],
        [-0.001, -0.005, -0.   ],
        [-0.   , -0.006,  0.   ],
        [-0.   ,  0.003, -0.   ],
        [-0.005, -0.   , -0.   ],
        [ 0.001, -0.005, -0.   ],
        [ 0.021, -0.035, -0.   ],
        [ 0.001, -0.004, -0.   ]]), array([[ 0.   ,  0.006,  0.   ],
        [ 0.   , -0.003, -0.   ],
        [ 0.005,  0.   , -0.   ],
        [-0.001,  0.005, -0.   ],
        [-0.021,  0.035, -0.   ],
        [-0.001,  0.004, -0.   ],
        [ 0.   , -0.206,  0.   ],
        [-0.   ,  0.115, -0.   ],
        [ 0.021,  0.035, -0.   ],
        [ 0.001,  0.004, -0.   ],
        [-0.005,  0.   , -0.   ],
        [ 0.001,  0.005, -0.   ]]))

In [23]:
AP.sites=[0,1,2,3,4,5,6,7,8,9,10,11]

In [24]:
AP.perturb()

In [25]:
AP.dV(0)

array([[-5.609, -0.699,  0.   , ..., -0.093,  0.   , -0.016],
       [-0.699, -0.865, -0.   , ..., -0.153,  0.   , -0.051],
       [ 0.   , -0.   , -0.858, ...,  0.153, -0.   ,  0.056],
       ...,
       [-0.093, -0.153,  0.153, ..., -0.374,  0.   , -0.084],
       [ 0.   ,  0.   , -0.   , ...,  0.   , -0.355, -0.   ],
       [-0.016, -0.051,  0.056, ..., -0.084, -0.   , -0.248]])

In [26]:
dV=DeltaV(Benz,[[0],[1]])

In [27]:
AP.mo1(2)

array([[ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,
         0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.

In [28]:
dV

array([[-5.609, -0.699,  0.   , ..., -0.093,  0.   , -0.016],
       [-0.699, -0.865, -0.   , ..., -0.153,  0.   , -0.051],
       [ 0.   , -0.   , -0.858, ...,  0.153, -0.   ,  0.056],
       ...,
       [-0.093, -0.153,  0.153, ..., -0.374,  0.   , -0.084],
       [ 0.   ,  0.   , -0.   , ...,  0.   , -0.355, -0.   ],
       [-0.016, -0.051,  0.056, ..., -0.084, -0.   , -0.248]])

In [29]:
rotate_matrix(dV,Benz,2,ref_site=0)-DeltaV(Benz,[[2],[1]])

array([[ 0.   ,  0.   ,  0.   , ..., -0.012, -0.   , -0.   ],
       [ 0.   ,  0.   ,  0.   , ..., -0.076, -0.   , -0.   ],
       [ 0.   ,  0.   ,  0.   , ..., -0.079,  0.   , -0.017],
       ...,
       [-0.012, -0.076, -0.079, ..., -0.012,  0.   ,  0.075],
       [-0.   , -0.   ,  0.   , ...,  0.   , -0.   ,  0.   ],
       [-0.   , -0.   , -0.017, ...,  0.075,  0.   , -0.   ]])

In [30]:
from AP_class import old_rotate_matrix

In [31]:
rotate_grad(Benz.atom_coords(),2,ref_site=0),Benz.atom_coords()

(array([[-2.262,  1.306,  0.   ],
        [-4.018,  2.32 ,  0.   ],
        [ 0.   ,  2.612, -0.   ],
        [ 0.   ,  4.639, -0.   ],
        [ 2.262,  1.306, -0.   ],
        [ 4.018,  2.32 , -0.   ],
        [ 2.262, -1.306,  0.   ],
        [ 4.018, -2.32 ,  0.   ],
        [-0.   , -2.612, -0.   ],
        [ 0.   , -4.639, -0.   ],
        [-2.262, -1.306, -0.   ],
        [-4.018, -2.32 ,  0.   ]]), array([[ 0.   ,  2.612, -0.   ],
        [ 0.   ,  4.639, -0.   ],
        [ 2.262,  1.306, -0.   ],
        [ 4.018,  2.32 , -0.   ],
        [ 2.262, -1.306,  0.   ],
        [ 4.018, -2.32 ,  0.   ],
        [-0.   , -2.612, -0.   ],
        [ 0.   , -4.639, -0.   ],
        [-2.262, -1.306, -0.   ],
        [-4.018, -2.32 ,  0.   ],
        [-2.262,  1.306,  0.   ],
        [-4.018,  2.32 ,  0.   ]]))

In [32]:
np.set_printoptions(precision=4)

In [33]:
pt=Benz.aoslice_by_atom()[0,-2]
rpt=Benz.aoslice_by_atom()[2,-2]

In [34]:
BenzHF.make_rdm1()[pt:pt+5,pt:pt+5],BenzHF.make_rdm1()[rpt:rpt+5,rpt:rpt+5]

(array([[ 2.0703, -0.2075, -0.    , -0.0018,  0.    ],
        [-0.2075,  0.7836,  0.    ,  0.0191,  0.    ],
        [-0.    ,  0.    ,  0.6088,  0.    ,  0.    ],
        [-0.0018,  0.0191,  0.    ,  0.6232, -0.    ],
        [ 0.    ,  0.    ,  0.    , -0.    ,  0.7854]]),
 array([[ 2.0703, -0.2075, -0.0016, -0.0009,  0.    ],
        [-0.2075,  0.7836,  0.0165,  0.0095, -0.    ],
        [-0.0016,  0.0165,  0.6196,  0.0062,  0.    ],
        [-0.0009,  0.0095,  0.0062,  0.6124,  0.    ],
        [ 0.    , -0.    ,  0.    ,  0.    ,  0.7854]]))

In [35]:
np.cos(np.pi/3)*0.0191,np.sin(np.pi/3)*0.0191

(0.009550000000000001, 0.016541085212282775)

In [36]:
np.array((0,0,1))

array([0, 0, 1])

In [37]:
coords=Benz.atom_coords()

In [38]:
#symm.Dmatrix.get_euler_angles(coords[0],coords[2])

In [39]:
#symm.geom.detect_symm(Benz.atom_and_coords())

In [40]:
AP.mo1(2).shape

(36, 21)

In [41]:
bsym=benz_Symm(Benz)

In [42]:
bsym.eqs

{2: {'ref': 0,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2278>},
 3: {'ref': 1,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd22b0>},
 4: {'ref': 0,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd22e8>},
 5: {'ref': 1,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2320>},
 6: {'ref': 0,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2358>},
 7: {'ref': 1,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2390>},
 8: {'ref': 0,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd23c8>},
 9: {'ref': 1,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2400>},
 10: {'ref': 0,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2438>},
 11: {'ref': 1,
  'op': <scipy.spatial.transform.rotation.Rotation at 0x7f87c8fd2470>}}

In [43]:
coords=Benz.atom_coords()

In [44]:
coords[::]

array([[ 0.    ,  2.6117, -0.    ],
       [ 0.    ,  4.6394, -0.    ],
       [ 2.2618,  1.3059, -0.    ],
       [ 4.0178,  2.3197, -0.    ],
       [ 2.2618, -1.3059,  0.    ],
       [ 4.0178, -2.3197,  0.    ],
       [-0.    , -2.6117, -0.    ],
       [ 0.    , -4.6394, -0.    ],
       [-2.2618, -1.3059, -0.    ],
       [-4.0178, -2.3197,  0.    ],
       [-2.2618,  1.3059,  0.    ],
       [-4.0178,  2.3197,  0.    ]])

In [45]:
rotate_grad(bsym.eqs[4]['op'].apply(coords),4,ref_site=0)

array([[ 0.    ,  2.6117, -0.    ],
       [ 0.    ,  4.6394,  0.    ],
       [ 2.2618,  1.3059,  0.    ],
       [ 4.0178,  2.3197,  0.    ],
       [ 2.2618, -1.3059, -0.    ],
       [ 4.0178, -2.3197, -0.    ],
       [-0.    , -2.6117, -0.    ],
       [-0.    , -4.6394, -0.    ],
       [-2.2618, -1.3059,  0.    ],
       [-4.0178, -2.3197,  0.    ],
       [-2.2618,  1.3059, -0.    ],
       [-4.0178,  2.3197, -0.    ]])

In [46]:
rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6)

No AF found for atom 0. Calculating it from code


array([[ 0.    ,  0.0131,  0.    ],
       [ 0.    , -0.0046, -0.    ],
       [-0.0014, -0.0081, -0.    ],
       [-0.0009,  0.0051, -0.    ],
       [ 0.0012, -0.0073, -0.    ],
       [ 0.0016,  0.0137, -0.    ],
       [ 0.    ,  0.0046,  0.    ],
       [-0.    , -0.0201, -0.    ],
       [-0.0012, -0.0073, -0.    ],
       [-0.0016,  0.0137,  0.    ],
       [ 0.0014, -0.0081, -0.    ],
       [ 0.0009,  0.0051, -0.    ]])

In [47]:
AP.af(6)

array([[ 0.    ,  0.0131,  0.    ],
       [ 0.    , -0.0046, -0.    ],
       [-0.0014, -0.0081, -0.    ],
       [-0.0009,  0.0051, -0.    ],
       [ 0.0012, -0.0073, -0.    ],
       [ 0.0016,  0.0137, -0.    ],
       [ 0.    ,  0.0046,  0.    ],
       [-0.    , -0.0201, -0.    ],
       [-0.0012, -0.0073, -0.    ],
       [-0.0016,  0.0137,  0.    ],
       [ 0.0014, -0.0081, -0.    ],
       [ 0.0009,  0.0051, -0.    ]])

In [48]:
np.allclose(rotate_grad(bsym.eqs[6]['op'].apply(AP.af(0)),6),AP.af(6))

True